In [ ]:
!pip install plotly
!pip install mplfinance

In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
import requests
import plotly.graph_objects as go
from datetime import datetime


Function to check for symbols

In [ ]:
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=tata&apikey=SHEYWPL1ZYCDZ17D'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
def fetch_daily_ohlc(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={apikey}"
    r = requests.get(url)
    return r.json()


In [ ]:
def calculate_percentage_differences(val1, val2):
    return abs(val1 - val2) / val2 * 100


def calculate_candle_length_percentage(open_price, close_price):
    return abs(close_price - open_price) / open_price * 100

In [ ]:
def is_marubozu(candle,
    candle_type="bullish",
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0):

    open_price = float(candle["1. open"])
    close_price = float(candle["4. close"])
    high_price = float(candle["2. high"])
    low_price = float(candle["3. low"])

    high_close_diff_pct = calculate_percentage_differences(high_price, close_price)
    open_low_diff_pct = calculate_percentage_differences(open_price, low_price)
    high_open_diff_pct = calculate_percentage_differences(high_price, open_price)
    close_low_diff_pct = calculate_percentage_differences(close_price, low_price)

    candle_length_pct = calculate_candle_length_percentage(open_price, close_price)

    if not (min_candle_length <= candle_length_pct <= max_candle_length):
        return False

    if candle_type == "bullish":
        return (
            high_close_diff_pct < upper_wick_threshold and
            open_low_diff_pct < lower_wick_threshold and
            close_price > open_price
        )
    elif candle_type == "bearish":
        return (
            high_open_diff_pct < upper_wick_threshold and
            close_low_diff_pct < lower_wick_threshold and
            open_price > close_price
        )
    else:
        raise ValueError("Invalid candle_type. Use 'bullish' or 'bearish'.")


def detect_marubozu(data,
    candle_type="bullish",
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0):
    time_series = data.get("Time Series (Daily)", {})
    results = []

    for date, candle in time_series.items():
        if is_marubozu(
            candle,
            candle_type,
            upper_wick_threshold,
            lower_wick_threshold,
            min_candle_length,
            max_candle_length
        ):
            results.append({
                "date": date,
                "type": candle_type,
                "open": float(candle["1. open"]),
                "high": float(candle["2. high"]),
                "low": float(candle["3. low"]),
                "close": float(candle["4. close"]),
                "volume": int(candle["5. volume"])
            })

    return results

In [ ]:
def detect_all_marubozus(data,
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0):

    bullish_results = detect_marubozu(
        data, "bullish",
        upper_wick_threshold,
        lower_wick_threshold,
        min_candle_length,
        max_candle_length
    )

    bearish_results = detect_marubozu(
        data, "bearish",
        upper_wick_threshold,
        lower_wick_threshold,
        min_candle_length,
        max_candle_length
    )

    # Add type label to each result dict
    for result in bullish_results:
        result['type'] = 'bullish'
    for result in bearish_results:
        result['type'] = 'bearish'

    combined_results = bullish_results + bearish_results

    return pd.DataFrame(combined_results)


In [ ]:
def plot_candlestick_with_marubozu(data, marubozu_df):
    # Extract the time series data and convert it into a DataFrame
    time_series = data.get("Time Series (Daily)", {})

    # Convert the time series into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert all values to float and rename columns for mplfinance compatibility
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    fig.show()

In [ ]:
apikey = "SHEYWPL1ZYCDZ17D"
symbol = "RELIANCE.BSE"
data = fetch_daily_ohlc(symbol, apikey)

marubozu_df = detect_all_marubozus(
        data,
        upper_wick_threshold=0.5,
        lower_wick_threshold=0.5,
        min_candle_length=1.0,
        max_candle_length=10.0
)

print("Detected Marubozu Candles:")
print(marubozu_df)


Detected Marubozu Candles:
          date     type     open     high      low    close   volume
0   2025-04-11  bullish  1200.00  1222.45  1197.05  1219.30   684519
1   2025-04-09  bullish  1172.40  1189.50  1169.10  1185.60   495957
2   2025-03-24  bullish  1286.40  1305.30  1281.00  1301.35   861533
3   2025-03-20  bullish  1254.20  1272.80  1250.00  1268.55   612659
4   2025-03-07  bullish  1214.05  1254.50  1212.30  1249.10   794738
5   2025-02-04  bullish  1247.05  1289.00  1246.65  1286.00   397483
6   2025-01-30  bullish  1235.00  1256.60  1232.25  1253.60   462230
7   2025-01-08  bullish  1251.20  1270.70  1245.00  1264.70   881916
8   2025-01-07  bullish  1221.65  1243.40  1221.40  1240.90   512057
9   2025-01-02  bullish  1220.70  1244.50  1220.25  1241.65   529144
10  2025-04-01  bearish  1274.60  1277.80  1250.00  1252.45   722008
11  2025-03-26  bearish  1290.25  1293.95  1269.00  1272.55   658347
12  2025-03-25  bearish  1307.00  1307.00  1283.00  1285.40   865836
13  202

In [ ]:
plot_candlestick_with_marubozu(data, marubozu_df)

In [ ]:
def calculate_profit_loss(data, marubozu_df):

    time_series = data.get("Time Series (Daily)", {})
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']


    df.index = pd.to_datetime(df.index)
    bullish_dates = pd.to_datetime(marubozu_df[marubozu_df['type'] == 'bullish']['date'])
    bullish_dates = bullish_dates[bullish_dates.isin(df.index)]
    total_profit_loss = 0
    trades = []

    for date in bullish_dates:
        try:

            buy_price = df.loc[date]['Close']
            next_trading_day = pd.bdate_range(date, periods=2)[1]

            if next_trading_day in df.index:

                sell_price = df.loc[next_trading_day]['Open']


                profit_loss = sell_price - buy_price
                total_profit_loss += profit_loss


                trades.append({
                    'buy_date': date,
                    'buy_price': buy_price,
                    'sell_date': next_trading_day,
                    'sell_price': sell_price,
                    'profit_loss': profit_loss
                })
        except KeyError:

            continue

    return total_profit_loss, trades

In [ ]:
total_profit_loss, trades = calculate_profit_loss(data, marubozu_df)

print(f"Total Profit/Loss: {total_profit_loss}")
for trade in trades:
    print(f"Buy on {trade['buy_date']} at {trade['buy_price']}, Sell on {trade['sell_date']} at {trade['sell_price']}, Profit/Loss: {trade['profit_loss']}")

Total Profit/Loss: 29.500000000000227
Buy on 2025-03-24 00:00:00 at 1301.35, Sell on 2025-03-25 00:00:00 at 1307.0, Profit/Loss: 5.650000000000091
Buy on 2025-03-20 00:00:00 at 1268.55, Sell on 2025-03-21 00:00:00 at 1270.05, Profit/Loss: 1.5
Buy on 2025-03-07 00:00:00 at 1249.1, Sell on 2025-03-10 00:00:00 at 1249.1, Profit/Loss: 0.0
Buy on 2025-02-04 00:00:00 at 1286.0, Sell on 2025-02-05 00:00:00 at 1284.95, Profit/Loss: -1.0499999999999545
Buy on 2025-01-30 00:00:00 at 1253.6, Sell on 2025-01-31 00:00:00 at 1257.4, Profit/Loss: 3.800000000000182
Buy on 2025-01-08 00:00:00 at 1264.7, Sell on 2025-01-09 00:00:00 at 1267.9, Profit/Loss: 3.2000000000000455
Buy on 2025-01-07 00:00:00 at 1240.9, Sell on 2025-01-08 00:00:00 at 1251.2, Profit/Loss: 10.299999999999955
Buy on 2025-01-02 00:00:00 at 1241.65, Sell on 2025-01-03 00:00:00 at 1247.75, Profit/Loss: 6.099999999999909
